# <u>__Prédiction de genres musicaux__</u>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Nous allons étudier un cas classique de classification multiclasse: Nous avons un dataset constitué de nombreux morceaux qui possèdent de nombreux caractéristiques tel qu'un nom, un artiste ou la popularité. L'objectif sera de determiné à partir de ces nombreuses variables le genre musicale de chacun de nos morceaux ('Electronic', 'Anime', nan, 'Jazz', 'Alternative', 'Country',
       'Rap', 'Blues', 'Rock', 'Classical', 'Hip-Hop').

In [ ]:
df = pd.read_csv('music_genre.csv')

df.head(5)

In [ ]:
df.columns

Nous avons reçu notre dataset 'propre' avec très peu de valeurs manquantes:

In [ ]:
print(f"Nous avons {df.isna().sum().sum()} valeurs manquantes sur {len(df)} valeurs")

Bien que nous ayons une notion de date avec la variable "obtained_date" qui est la variable associée à la date d'obtention, nous n'étudions pas de série temporelle, supprimé les lignes avec des valeurs manquantes ne nous pose donc pas de problème, nous remarquons que le tempo possède des éléments string "?" sûrement utilisé à place du Nan par celui qui a fournit le dataset sur kaggle, que nous ne pouvons pas voir avec isna(). Nous les remplaçons par la moyenne des autres valeurs qui appartiennent à un mêmes groupe de musique, car le tempo devrait être similaire pour des musiques du mêmes genre ou du moins suivre une tendance.

In [ ]:
df.dtypes

In [ ]:
df=df.dropna()
#On transforme tous les '?' en nan:
df['tempo'] = pd.to_numeric(df['tempo'],errors='coerce')

print(f"On a donc {df['tempo'].isna().sum()} Nan values après avoir transformé les '?'")

for genre in df["music_genre"].unique():
    df.loc[(df["music_genre"]==genre)*df["tempo"].isna(),'tempo'] = df.loc[(df["music_genre"]==genre)*(~ df["tempo"].isna()),'tempo'].mean()

print(df['tempo'].isna().sum())

In [ ]:
float('115.00200000000001')

De plus nous avons deux variables qualitatives "mode", "key", pour pouvoir les utiliser lors de l'apprentissage, nous créeons des indicatrices lors de la présence ou non d'un élément spécifique pour chacune des valeurs que peuvent prendre les variables:

In [ ]:
key_elements = df["key"].unique()
print(f"Les différents éléments de 'key' sont: { key_elements }")

In [ ]:
for j in key_elements:
    df[j]=(df["key"]==j)*1

In [ ]:
mode_elements = df["mode"].unique()
print(f"Les différents éléments de 'mode' sont: {mode_elements}") 

In [ ]:
for j in mode_elements:
    df[j]=(df["mode"]==j).astype(int)

In [ ]:
df=df.drop(columns=["key","mode"])
df.columns

Mis à part les variables 'instance_id', 'artist_name', 'track_name' qui nous permettent de nommer nos données, la variable "obtained_date" est inutile pour la classification car nous avons 5 dates qui n'apportent pas d'informations particulière:

In [ ]:
df["obtained_date"].unique()

Pour la classification nous nous concentrons sur les variables suivantes: 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'music_genre', 'A#', 'D', 'G#', 'C#', 'F#',
       'B', 'G', 'F', 'A', 'C', 'E', 'D#', 'Minor', 'Major'.

Bien sûr le choix des variables les plus importantes dépend du type de modèle de classification utilisée.

On divise notre dataset en base de donnée test et d'entrainement (20% et 80%) de manière qu'il y est un nombre equivalent de genre pour chacune des bases d'entraînement:

In [ ]:
from sklearn.model_selection import train_test_split

X = df[['popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'A#', 'D', 'G#', 'C#', 'F#',
       'B', 'G', 'F', 'A', 'C', 'E', 'D#', 'Minor', 'Major']]

y = df['music_genre']
for j, value in enumerate(y.unique()):
    print(f"{value} est la valeur {j}")
    df.loc[y==value,"token_class"] = j

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

---

<div style="text-align:center;">
<h1>PARTIE ISSAM</h1>
</div>  

---

#### __I - Modèle de Regression logistique.__

In [ ]:
import seaborn as sns

correlation_matrix = df[["token_class",'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'A#', 'D', 'G#', 'C#', 'F#',
       'B', 'G', 'F', 'A', 'C', 'E', 'D#', 'Minor', 'Major',]].corr()

# Create the heatmap
plt.figure(figsize = (10,8))
sns.heatmap(correlation_matrix, cmap = 'coolwarm')
plt.show()

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import accuracy_score

clf = LR(multi_class="multinomial",n_jobs=-1).fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

#### __II - Linear Discriminent Analysis (LDA).__

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

---

<div style="text-align:center;">
<h1>PARTIE ABDOULAYE</h1>
</div>  

---

---

<div style="text-align:center;">
<h1>PARTIE LUCAS</h1>
</div>  

---